# Knn evaluation

In [13]:
import os
import sys
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import datasets
from torchvision import transforms as pth_transforms
from torchvision import models as torchvision_models

import utils
import vision_transformer as vits

from eval_knn2 import extract_feature_pipeline, knn_classifier

In [19]:
args = {}
args["batch_size_per_gpu"] = 2
args["nb_knn"] = 5
args["temperature"] = 0.07
args["pretrained_weights"] = ""
args["use_cuda"] = False
args["arch"] = "vit_small"
args["patch_size"] = 16
args["checkpoint_key"] = "teacher"
args["dump_features"] = None
args["load_features"] = None
args["num_workers"] = 4
args["dist_url"] = "env://"
args["local_rank"] = 0
args["data_path"] = "./data/tiny_imagenet/tiny-imagenet-200/"
args["rank"] = 0
args["gpu"] = 0
args["world_size "] = 1

In [20]:
print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(args.items())))
cudnn.benchmark = True

arch: vit_small
batch_size_per_gpu: 2
checkpoint_key: teacher
data_path: ./data/tiny_imagenet/tiny-imagenet-200/
dist_url: env://
dump_features: None
gpu: 0
load_features: None
local_rank: 0
nb_knn: 5
num_workers: 4
patch_size: 16
pretrained_weights: 
rank: 0
temperature: 0.07
use_cuda: False
world_size : 1


In [21]:
if args["load_features"]:
    train_features = torch.load(os.path.join(args["load_features"], "trainfeat.pth"))
    test_features = torch.load(os.path.join(args["load_features"], "testfeat.pth"))
    train_labels = torch.load(os.path.join(args["load_features"], "trainlabels.pth"))
    test_labels = torch.load(os.path.join(args["load_features"], "testlabels.pth"))
else:
    # need to extract features !
    train_features, test_features, train_labels, test_labels = extract_feature_pipeline(args)


FileNotFoundError: [WinError 3] The system cannot find the path specified: './data/tiny_imagenet/tiny-imagenet-200/val'

In [ ]:
if utils.get_rank() == 0:
    if args["use_cuda"]:
        train_features = train_features.cuda()
        test_features = test_features.cuda()
        train_labels = train_labels.cuda()
        test_labels = test_labels.cuda()

    print("Features are ready!\nStart the k-NN classification.")
    for k in args["nb_knn"]:
        top1, top5 = knn_classifier(train_features, train_labels,
            test_features, test_labels, k=k, T=args["temperature"], num_classes=200)
        print(f"{k}-NN classifier result: Top1: {top1}, Top5: {top5}")
dist.barrier()